In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [1]:
from transformers import pipeline

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    '../hf-models/usvsnsp-sft-combined',
    device_map = 'auto',
    cache_dir = '../hf-models',
    load_in_8bit = True
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /mnt/sairam/env/lib/python3.7/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /mnt/sairam/env/lib/python3.7/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/mnt/sairam/env/lib/python3.7/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /opt/conda did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
This model has some weights that should be kept in higher precision, you need to upgrade `accelerate` to properly deal with them (`pip install --upgrade accelerate`).


In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    '../hf-models/usvsnsp-sft-model', 
    cache_dir = '../hf-models',
#     additional_special_tokens = (
#         '<|query|>',
#         '</query/>',
#         '<|response|>',
#         '</response/>',
#         '<|result|>',
#         '</result/>',
#         '<|summary|>',
#         '</summary/>',
#         '<|results|>',
#         '</results/>',
#         '<|result-sentence|>'
#     )
)

In [10]:
def generate(text):
    tokens = tokenizer([text], return_tensors = 'pt').input_ids.cuda()
    text = model.generate(
        input_ids = tokens, 
        temperature = 0.8, 
        top_k = 40,
        max_length = 128,
        repetition_penalty = 2.0,
    )
    print(tokenizer.decode(text[0]))

In [11]:
generate("<|prompt|>Hey I am a farmer but I am not able to understand what to do to stop mango leaves from falling.</prompt/><|summary|>")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [6]:
pipe = pipeline('text-generation', model = model, tokenizer = tokenizer, device = 3)

In [9]:
pipe(
    "Hey I am a farm", 
    max_length=128,
)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': "Hey I am a farmer but I am not a farmer. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer's son. I am a farmer"}]

In [10]:
with open("../kcc_data/jsonlines/data.jsonl", 'r') as reader:
        data = [i[1:-2] for i in reader]

In [11]:
tokens = tokenizer(
    [data[0]], 
    return_tensors = 'pt',
    truncation = True,
    max_length = 2048
)

In [12]:
input_ids = tokens['input_ids'][:, 0:-1].cuda()
labels = tokens['input_ids'][:, 1:].cuda()

In [ ]:
model = 

In [13]:
input_ids

tensor([[   29,    93,  7267,  ...,   964,   870, 10927]])

In [14]:
labels

tensor([[   93,  7267, 49651,  ...,   870, 10927,  4357]])